# HW4 -- Diamonds
## Due Sunday May 21, at 11:55pm

Jennifer MacDonald 604501712

CS249 -- Spring 2017 -- D.S. Parker &copy; 2017

## Assignment Overview

The goal of this assignment is for you to develop models for the <a href="http://docs.ggplot2.org/current/diamonds.html"><b>diamonds</b> dataset</a>,
which is included in the <a href="http://ggplot2.org">ggplot2</a> package.

This is a very simple assignment:  you are asked to build four models:
LDA or QDA, simple Linear Regression, log-scaled Linear Regression, and Logistic Regression.
You then just upload the formulas (R commands) you used to construct these models to CCLE.


# Generation of the numeric.diamonds Dataset

## Transformed dataset for building all Models

In [311]:
# we need the ggplot2 package to get the "diamonds" dataset

not.installed <- function(pkg) !is.element(pkg, installed.packages()[,1])

if (not.installed("ggplot2")) install.packages("ggplot2")
library(ggplot2)
    
# library(devtools)
# install_github("ggbiplot", "vqv")
     
data(diamonds, package="ggplot2") 

# load the MASS package, which includes simple implementations of LDA and QDA

not.installed <- function(pkg) !is.element(pkg, installed.packages()[,1])

if (not.installed("MASS"))  install.packages("MASS")  # we need the MASS package
library(MASS)  #  load the MASS package   

# Measuring Accuracy of Models in this Assignment

## Accuracy of an LDA or QDA model is the percentage of correct classifications

In [312]:
classification_accuracy = function( model, test.data, test.solutions ) {
    # The value is a vector or matrix of predicted values corresponding to the determining variable values in 
    # data.frame
    predictions = predict( model, newdata = test.data )
    
    # check whether the predicted values in "cut" are the same as the actual values
    incorrect.predictions  =  (predictions$class != test.solutions )
    
    # get the place and value of the incorrect predictions
    incorrect.ids = test.solutions[incorrect.predictions]
    
    # subtract the incorrect values from the total length and divide by the length of the total to get the percentage
    # or correct classification
    accuracy = (length(test.solutions) - length(incorrect.ids)) / length(test.solutions)
    
    return (accuracy)
}

## Accuracy of a Linear Regression model is its R<sup>2</sup> value

In [313]:
linear_regression_accuracy = function( model, test.data, test.solutions ) {
    predictions = predict( model, newdata = test.data )
    
    # Extract the regression coefficient (matrix)
    intercept = coef(model)["(Intercept)"] 
    
    # calculate r-squared
    accuracy = Rsquared(test.solutions, predictions, intercept)
    
    return (accuracy) 
}

Rsquared = function (y, yhat, intercept) {
    RSS = sum((y-yhat)^2) # residual sum of squares
    if (intercept) { MSS = sum((yhat-mean(yhat))^2) } # model has an intercept
    else { MSS = sum((yhat)^2) } # model does not have an intercept
    return(MSS/(MSS + RSS)) # return muliple r-squared
}

## Accuracy of a Logistic Regression model is the percentage of correct classifications

In [314]:
logistic_regression_accuracy = function( model, test.data, test.solutions ) {
    predictions = predict( model, newdata = test.data, type = "response") 
    PredictionClasses = round(predictions) # Turn the numeric values into {0,1} values
    
    incorrect.predictions = (PredictionClasses != test.solutions)
    
    # Matrix charting performance of classifications 
    confusion.matrix = table( PredictionClasses, test.solutions ) 
    
    accuracy = sum(diag(confusion.matrix)) / nrow(test.data)
    
    return (accuracy)
}


## Basic cleaning of the data

In [315]:
diamonds = subset( diamonds, (x>0) & (y>0) & (z>0) )  #  There are actually some zero values, we omit them.

### Convert the categorical values to integers -- using the unclass() function.

In [316]:
numeric.diamonds = transform( diamonds,
                              cut = as.numeric(unclass(diamonds$cut)),
                              color = as.numeric(unclass(diamonds$color)),
                              clarity = as.numeric(unclass(diamonds$clarity))
                   )

### Build a training set and test set (as subsets of numeric.diamonds) -- using your UID

In [317]:
MY_UID = 604501712 ########## you must enter your UCLA UID here !!!
set.seed( MY_UID )
n = nrow( numeric.diamonds )
sample.size = 0.75 * n   ###### Use 75% of the data for the training set
training.row.ids = sample( (1:n), sample.size )

my.training.set = numeric.diamonds[  training.row.ids, ]
my.test.set     = numeric.diamonds[ -training.row.ids, ]   # set complement of training.set.ids

### Compute accuracy of 4 Baseline Models about diamonds

In [318]:
# a QDA classification model that predicts a diamond's Cut
baseline_m1 = qda( cut ~ .,           data=my.training.set )
baseline_m1_accuracy = classification_accuracy(baseline_m1, my.test.set, my.test.set$cut)

# a linear regression model that predicts a diamond's Price
baseline_m2 = lm(  price ~ .,         data=my.training.set )
baseline_m2_accuracy = linear_regression_accuracy( baseline_m2, my.test.set, my.test.set$price )

# a linear regression model that predicts a diamond's log10(Price)
baseline_m3 = lm(  log10(price) ~ .,  data=my.training.set )
baseline_m3_accuracy = linear_regression_accuracy( baseline_m3, my.test.set, log10(my.test.set$price) )

# a logistic regression model that predicts whether a diamond's Price is above $1500
baseline_m4 = suppressWarnings( glm( I(price>1500) ~ ., data=my.training.set, family=binomial ))
baseline_m4_accuracy = logistic_regression_accuracy( baseline_m4, my.test.set, I(my.test.set$price>1500) )

### Build 4 Models improving on the Baseline Models

In [319]:
#  The Baseline models:

m1 = qda( cut ~ price + table + color + clarity,       data=my.training.set )
m1_accuracy = classification_accuracy(m1, my.test.set, my.test.set$cut)

m2 = lm(  price ~ (carat + cut + clarity + color)^2,         data=my.training.set )
m2_accuracy = linear_regression_accuracy(m2, my.test.set, my.test.set$price)

m3 = lm(  log10(price) ~ (carat * cut * clarity * color * x * y * z)^2, data=my.training.set )
m3_accuracy = linear_regression_accuracy(m3, my.test.set, log10(my.test.set$price))

m4 = suppressWarnings( glm( I(price>1500) ~ carat * cut * clarity * color * x * y * z,     data=my.training.set, family = binomial ))
m4_accuracy = logistic_regression_accuracy(m4, my.test.set, I(my.test.set$price>1500))

### Print the accuracy of the Baseline and Improved Models

In [320]:
cat(baseline_m1_accuracy, ", qda( cut ~ .,           data=my.training.set )\n")
cat(baseline_m2_accuracy, ", lm(  price ~ .,         data=my.training.set )\n")
cat(baseline_m3_accuracy, ", lm(  log10(price) ~ .,  data=my.training.set )\n")
cat(baseline_m4_accuracy, ", glm( I(price>1500) ~ ., data=my.training.set, family=binomial )\n")

cat(m1_accuracy, ", qda( cut ~ price + table + color + clarity,                           data=my.training.set )\n")
cat(m2_accuracy, ", lm( price ~ (carat + cut + clarity + color )^2 ),                     data=my.training.set )\n")
cat(m3_accuracy, ", lm(  log10(price) ~ (carat * cut * clarity * color * x * y * z )^2 ), data=my.training.set )\n")
cat(m4_accuracy, ", glm( I(price>1500) ~ carat * cut * clarity * color * x * y * z, )     data=my.training.set, family = binomial )")

0.5537834 , qda( cut ~ .,           data=my.training.set )
0.9078241 , lm(  price ~ .,         data=my.training.set )
0.9768911 , lm(  log10(price) ~ .,  data=my.training.set )
0.9798961 , glm( I(price>1500) ~ ., data=my.training.set, family=binomial )
0.603635 , qda( cut ~ price + table + color + clarity,                           data=my.training.set )
0.9283409 , lm( price ~ (carat + cut + clarity + color )^2 ),                     data=my.training.set )
0.9861304 , lm(  log10(price) ~ (carat * cut * clarity * color * x * y * z )^2 ), data=my.training.set )
0.9860534 , glm( I(price>1500) ~ carat * cut * clarity * color * x * y * z, )     data=my.training.set, family = binomial )